In [1]:
import sys
sys.path.append('third_party/Matcha-TTS')
from cosyvoice.cli.cosyvoice import CosyVoice, CosyVoice2
from cosyvoice.utils.file_utils import load_wav
import torchaudio
from IPython.display import Audio
import os
import torch

# cosyvoice = CosyVoice2('pretrained_models/CosyVoice2-0.5B', load_jit=False, load_trt=False, fp16=False)
cosyvoice = CosyVoice2('/data/minju/TTS/CosyVoice/pretrained_models/CosyVoice2-0.5B', load_jit=True, load_trt=True, fp16=False)
#cosyvoice = CosyVoice('./pretrained_models/CosyVoice-300M',load_jit=False, load_trt=False)

2025-03-21 15:01:57,695 - modelscope - INFO - PyTorch version 2.0.0 Found.
2025-03-21 15:01:57,697 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2025-03-21 15:01:57,856 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 13b7082244438de82b16012a4fd7baad and a total number of 980 components indexed


failed to import ttsfrd, use WeTextProcessing instead


/data/minju/conda/envs/cosyvoice/lib/python3.10/site-packages/diffusers/models/lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)
2025-03-21 15:02:05,735 INFO input frame rate=25
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/data/minju/conda/envs/cosyvoice/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
2025-03-21 15:02:10,375 WETEXT INFO building fst for zh_

[03/21/2025-15:03:10] [TRT] [I] Loaded engine size: 290 MiB
[03/21/2025-15:03:11] [TRT] [E] 1: [defaultAllocator.cpp::allocate::19] Error Code 1: Cuda Runtime (out of memory)
[03/21/2025-15:03:11] [TRT] [W] Requested amount of GPU memory (9170563584 bytes) could not be allocated. There may not be enough free memory for allocation to succeed.
[03/21/2025-15:03:11] [TRT] [E] 2: [executionContext.cpp::ExecutionContext::552] Error Code 2: OutOfMemory (no further information)


In [3]:
import time
refer_audio = '/data/minju/TTS/CosyVoice/refer/men_love_refer.wav'
refer_audio = '/data/minju/TTS/CosyVoice/refer/men_love_refer_02.wav'
refer_audio = '/data/minju/TTS/CosyVoice/refer/tony_default_0420_base_00019.wav'
refer_name = refer_audio.split("/")[-1].replace(".wav", "")
print(refer_name)
prompt_speech_16k = load_wav(refer_audio, 16000)
tts_text = "오늘은 대기 정체와 국외 미세먼지까지 유입되면서 공기 질이 나쁘겠는데요."
tts_text = "선수 시절뿐 아니라 각종 선거에서 이변을 일으켜온 그답게 이번에도 그 진가를 보여줬다. 이번 체육회장 선거는 이기흥 현 회장 당선 가능성이 높다는 예상이 지배적이었다. 선거인단 구성 자체도 이 회장에게 유리하다는 지적이 있었다. 추첨으로 뽑히는 인원 외에 전국 이백이십팔개 시 군 구 체육회에서 추천한 인사가 선거인단에 포함되는 지정 선거인 제도가 도입됐는데 현직 회장으로 지역 체육회와 접촉이 많았던 이 회장을 위한 제도라는 비판이 있었다. 실제 이날 투표 현장에선 유 후보가 당선되자 환호성을 지르는 젊은 체육인들이 대거 눈에 띄었다. 대한체육회 전직 임원은 젊은 체육인들이 변화를 원했다. 체육인들을 만나보면 한국 체육계가 나이 드신 분들이 권위를 내세워 끌고 간다는 비판이 많았다. 젊은 사람이 와서 변화를 이끌어줬으면 하는 바람이 많았다고 전했다."
tts_text = "<|en|>I have never been in a relationship with my ideal type."
tts_text = "<|jp|>理想型に恋愛するただ一度もやったことはありません。"
tts_text = "<|jp|>私は理想のタイプと付き合ったことがない。"
# tts_text = "<|zh|>我从来没有经历过一段符合我理想类型的关系。"
tts_text = '<|en|>There is a hypothesis that the hippocampus, a brain region that plays a key role in memory formation, is not fully developed in infants.'
# prompt_text = '손석희 전 앵커와 진행한 퇴임전 마지막 언론 대담 인터뷰 내일부터 이틀간 방송을 앞두고 있습니다.'
# prompt_text = '이상형에 부합하는 연애 단 한번도 해본적 없습니다.'

save_path = f'output-vc/250320-men-love'
os.makedirs(save_path, exist_ok=True)
total_tensor = []
start = time.time()
for i, j in enumerate(cosyvoice.inference_cross_lingual(tts_text, prompt_speech_16k, stream=False)):
    total_tensor.append(j['tts_speech'])
    #torchaudio.save(f'{save_path}/{refer_name}_{i}_{tts_text[:3]}.wav'.format(i), j['tts_speech'], cosyvoice.sample_rate)
end = time.time()
print(end-start)
concatenated_tensor = torch.cat(total_tensor, dim=-1)
torchaudio.save(f'{save_path}/{refer_name}_f_{tts_text[:3]}.wav', concatenated_tensor, cosyvoice.sample_rate)
Audio(f'{save_path}/{refer_name}_f_{tts_text[:3]}.wav')

tony_default_0420_base_00019


  0%|          | 0/1 [00:00<?, ?it/s]

update! <|en|> There is a hypothesis that the hippocampus, a brain region that plays a key role in memory formation, is not fully developed in infants.
33
update! 
0
max value is  tensor(1.0034)


2025-03-21 15:03:22,677 INFO synthesis text <|en|> There is a hypothesis that the hippocampus, a brain region that plays a key role in memory formation, is not fully developed in infants.
  0%|          | 0/1 [00:25<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'set_input_shape'